In [ ]:
import numpy as np
import pandas as pd
import os
import math
import time
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn import metrics
import matplotlib.pyplot as plt
from pyecharts import Line, Scatter, Bar, Overlap, Timeline, HeatMap, ThemeRiver, Grid, Page
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from lightgbm.sklearn import LGBMClassifier # LightGBM
from sklearn.linear_model import LogisticRegression
from ipywidgets import interact, fixed

from scipy.stats import spearmanr
import heapq
import copy
from tqdm import tnrange,tqdm_notebook
from time import sleep

import scipy.io as sio

In [ ]:
read_path=r'C:\Users\DYT\Desktop\Data\data_system\stock\daily\20180425'
factor_path = r'C:\Users\DYT\Desktop\Data\data_system\stock\ml_factor180614new'
factor_pathold = r'C:\Users\DYT\Desktop\Data\data_system\stock\ml_factor'
######### read .mat file in "data" directory ########
def read_data(name,path=read_path):
    path = path + '\\' + name
    return sio.loadmat(path)[name]

In [ ]:
valid_st_stock = read_data('valid_st_stock')
ipo_date = read_data('ipo_date')
stock_rt = read_data('stock_rt', path=factor_path)
gfn1_code_index = read_data('gfn1_code_index')
trd_dates = read_data('trd_dates')

In [ ]:
# obtain 161 factor names as lsit factor_names_old
# obtain new 10 factors as ml_zs_names
# there are 171 factors in total and the whole list is named as ml_names_all
factor_names_old = []
col_vec = []
for info in os.listdir(factor_pathold):
    index = info.rfind('.')
    name = info[:index]
    factor_names_old.append(name)

factor_names_new = []
col_vec = []
for info in os.listdir(factor_path):
    index = info.rfind('.')
    name = info[:index]
    factor_names_new.append(name)

factor_names_new.remove('stock_rt')

ml_zs_names = []
ml_bk_names = []
ml_bk_rt_names = []
for info in factor_names_new:
    for sufix in ['zs','bk','rt']:
        if info[-2:] == 'zs':
            ml_zs_names.append(info)
        elif info[-2:] == 'bk':
            ml_bk_names.append(info)
        elif info[-2:] == 'rt':
            ml_bk_rt_names.append(info)
            
factor_new_ml_names = sorted(list(set(factor_names_new).
                                  difference(set(ml_zs_names + 
                                                 ml_bk_names + 
                                                 ml_bk_rt_names))))
ml_zs_names = sorted(np.unique(np.array(ml_zs_names)).tolist())
ml_bk_names = sorted(np.unique(np.array(ml_bk_names)).tolist())
ml_bk_rt_names = sorted(np.unique(np.array(ml_bk_rt_names)).tolist())

factor_names_old = sorted(factor_names_old)
ml_zs_names = sorted(ml_zs_names)

factor_names_old.remove('falpha5')
factor_names_old.remove('falpha36')
factor_names_old.remove('falpha51')
factor_names_old.remove('falpha11')
ml_names_all = sorted(factor_names_old + ml_zs_names)


for info in factor_names_old:
    globals()[info] = read_data(info, path=factor_pathold)
    
for info in ml_zs_names:
    globals()[info] = read_data(info, path=factor_path)

In [ ]:
modmat = np.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
                   [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
                   [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
                   [0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
                   [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                   [0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
                   [0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
                   [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
                   [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

def zs_bk1(arr):#将z_score矩阵化为分类矩阵的函数
    starttime = time.time()
    
    arrvec = []
    for i in range(len(arr[0])):
        d_len = len(arr[:, i])-len(np.where(np.isnan(arr[:, i]))[0])
        remainder = d_len % 10
        numorg = d_len // 10
        condi = (modmat[:, remainder] + numorg).cumsum()
        rankmat = pd.DataFrame(arr[:, i]).rank(ascending=False, 
                                               method='min').values
        for j in range(10):
            if j == 0:
                rankmat[rankmat <= condi[j]] = j+1
            else:
                rankmat[(rankmat <= condi[j]) & (rankmat > condi[j-1])] = j+1
        arrvec.append(rankmat)
    arrmat = np.concatenate(arrvec,axis=1)
    endtime= time.time()
    t = endtime - starttime
#     print('time_last: {} min'.format(t/60))
    return arrmat

def bk_rt1(bk_arr):#将分类矩阵化为按类收益的矩阵
    starttime=time.time()
    rtr_row = []
    for i in range(len(bk_arr[0])-1):
        rtr_col = []
        for j in range(1, 11):
            cod_ind = np.where(bk_arr[:, i] == j)[0]
            bk_rtr_per = pd.DataFrame(stock_rt[cod_ind, i+1]).sum()[0]
            nna = len(cod_ind)
            bk_rtr_avg = bk_rtr_per/nna
            rtr_col.append(bk_rtr_avg)
        rtr_row.append(np.array(rtr_col))
    bkrtmat = np.concatenate(rtr_row).reshape(len(bk_arr[0])-1, 10).T
    endtime = time.time()
#     print('timelast: {} min'.format((endtime-starttime)/60))
    return bkrtmat

In [ ]:
stock_rt_bk = zs_bk1(stock_rt)#把股票收益分类

#把所有因子矩阵化为bk_rt矩阵
for info in ml_names_all:
    arr = globals()[info]
    globals()[info+'_bk_rt_p'] = bk_rt1(zs_bk1(arr))

#求所有因子的准确率的list，一个因子对应一个list，list里就是每天对应的准确率
for info in ml_names_all:
    globals()['accu_'+info] = []
    ml_bk = zs_bk1(globals()[info])
    for i in range(len(stock_rt[0])-1):
        concatvec = np.concatenate([ml_bk[:,i], 
                                    stock_rt_bk[:,i+1]]).reshape(2,len(stock_rt)).T
        concatvec = pd.DataFrame(concatvec).dropna().values
        accu = metrics.accuracy_score(concatvec[:,0], concatvec[:,1])
        globals()['accu_'+info].append(accu)

#求所有因子的 与未来一天收益的 IC值list，一个因子对应一个list，list里就是每天对应的IC值
for info in ml_names_all:
    globals()['corr_'+info] = []
    ml_zs = globals()[info]
    for i in range(len(stock_rt[0])-1):
        concatvec = np.concatenate([ml_zs[:,i], 
                                    stock_rt[:,i+1]]).reshape(2,len(stock_rt)).T
        concatvec = pd.DataFrame(concatvec).dropna().values
        corr = spearmanr(concatvec[:,0],concatvec[:,1]).correlation
        globals()['corr_'+info].append(corr)

In [ ]:
def slide20(data):
    return pd.DataFrame(data).rolling(window=20,center=False).mean().iloc[:,-1].values

def cumsum_max(arr, fa_t, fa_len):
# 根据上面的bk_rt求时间区间内的累积收益（做了滑动平均）的最大值。时间区间是根据fa_t, fa_len确定的
# fa_t即之后机器学预测的某天的前一天的值，fa_len即取fa_t的前多少天用于计算累积收益、准确率和IC值
    cum_arr = pd.DataFrame(arr[0, fa_t-fa_len+1:fa_t+1]).cumsum().values.T[0]#做多未做空
    sld20_cum_arr = pd.DataFrame(slide20(list(cum_arr)))
    arr_max = sld20_cum_arr.max()[0]  
    return arr_max

def accu_avg(accu_ml_l, fa_t, fa_len):#求相应时间区间内的准确率（做了滑动平均）的平均值
    sld20_accu_ml_df = pd.DataFrame(slide20(accu_ml_l[fa_t-fa_len:fa_t]))
    sld20_mean = sld20_accu_ml_df.mean()[0]    
    return sld20_mean

def corr_avg(corr_ml_l, fa_t, fa_len):#求相应时间区间内的IC值（做了滑动平均）的平均值
    sld20_corr_ml_df = pd.DataFrame(slide20(corr_ml_l[fa_t-fa_len:fa_t]))###
    sld20_mean = sld20_corr_ml_df.mean()[0]#[0]      
    return sld20_mean

In [ ]:
def three_condi_arr(fa_len, fa_t):
#这个函数是求出相应时间区间内的三个arr（类似于list的形式），比如max_arr就是171个因子在样本区间内的累积收益滑动平均最大值
    max_list = []
    accu_describe_list = []
    corr_describe_list = []
    for info in ml_names_all:
        arr = globals()[info+'_bk_rt_p']
        max_list.append(cumsum_max(arr, fa_t, fa_len))
        
        accu_ml_l = globals()['accu_'+info]
        accu_describe_list.append(accu_avg(accu_ml_l, fa_t, fa_len))
        
        corr_ml_l = globals()['corr_'+info]
        corr_describe_list.append(corr_avg(corr_ml_l, fa_t, fa_len))
        
    max_arr = np.array(max_list)
    accu_arr = np.array(accu_describe_list)
    corr_arr = np.array(corr_describe_list)

    return max_arr, accu_arr, corr_arr

def factor_acr_t(intersc_k, max_arr, accu_arr, corr_arr):
#将去掉重复因子名字的max_arr, accu_arr, corr_arr剩余因子相应的intersc_k即取排名前多少的股票做交集 
    max_rank_arr = pd.DataFrame(max_arr).rank(ascending=False, 
                                              method='min').values.T[0]
    accu_rank_arr = pd.DataFrame(accu_arr).rank(ascending=False, 
                                                method='min').values.T[0]
    corr_rank_arr = pd.DataFrame(corr_arr).rank(ascending=False, 
                                                method='min').values.T[0]
    fac_condi = np.concatenate([max_rank_arr, 
                                accu_rank_arr, 
                                corr_rank_arr]).reshape(3,len(ml_names_all)).T
    
    cumrt_idx = np.where(fac_condi[:,0] <= intersc_k)[0]
    accu_idx = np.where(fac_condi[:,1] <= intersc_k)[0]
    corr_idx = np.where(fac_condi[:,2] <= intersc_k)[0]
    
    condi_idx = list(set(cumrt_idx).intersection(set(accu_idx), set(corr_idx)))
    condi_names = [ml_names_all[i] for i in condi_idx]
    return sorted(condi_names)

def mlnames_func(fa_len, fa_t, intersc_k):
#这个函数就是把这个cell里前面几个函数逐一顺序拼起来
    max_arr, accu_arr, corr_arr = three_condi_arr(fa_len, fa_t)
    ml_names = factor_acr_t(intersc_k, max_arr, accu_arr, corr_arr)
    return ml_names

In [ ]:
fa_len = 240
intersc_k = 50#每个条件取前30求交集

ml_names_listog = []
stime = time.time()
for fa_tf in range(fa_len, len(stock_rt[0])):#2595是20170901，2675是20171229
    ml_names_t = mlnames_func(fa_len, fa_tf, intersc_k)
    ml_names_listog.append(ml_names_t)
etime = time.time()
print('timelast {} min'.format((etime-stime)/60))

In [ ]:
ml_names_list = [[] for i in range(fa_len)]+ml_names_listog#保证这个list跟全时期长度一样

In [ ]:
def vad_index(t):#去除st,退市股票和无行业股票后剩下的股票的有效索引
    st_indx = (np.where(valid_st_stock[:, t] == 1))[0]

    ipo_date[ipo_date <= trd_dates[t]] = 1
    ipo_date[ipo_date > trd_dates[t]] = 0
    ipo_indx = np.where(ipo_date == 0)[0]

    class_na_indx = np.where(np.isnan(gfn1_code_index[:, t]))[0]

    stock_indx = np.arange(stock_rt.shape[0])

    valid_indx = list(set(stock_indx).
                      difference(set(ipo_indx).union(set(class_na_indx), 
                                                     set(st_indx))))

    return valid_indx

In [ ]:
def fa_rank_rtbk(start_t, end_t):
#进行排名
#输入样本期起始天和要预测的那天，即样本期末尾那天的后一天，输出一个list，list里有相应样本期长度对应个数的XY数组，及要预测那天的有效股票索引
    fa_total = []
    for t in range(start_t, end_t+1):
        fa_cmb = []
        vad_id = vad_index(t)
        for info in ml_names_list[end_t-1]:      
            fa_vec = globals()[info][vad_id, t]
            fa_vec_df = pd.DataFrame(fa_vec).rank(ascending=False, method='dense')
            fa_vec_arr = fa_vec_df.values
            nna_len = len(fa_vec_df.dropna())
            fa_vec_rank_arr = fa_vec_arr / len(fa_vec_df.dropna())
            fa_cmb.append(fa_vec_rank_arr.T[0])          
        fa_cmb.append(stock_rt[vad_id, t+1])
        
        fa_cmb_cc = np.concatenate(fa_cmb).reshape((len(ml_names_list[end_t-1])+1), len(vad_id)).T
        if t == end_t:
            na_idx = np.unique(np.where(np.isnan(fa_cmb_cc))[0])
            nna_idx = sorted(list(set(np.arange(len(vad_id))).difference(set(na_idx))))
            nna_idx_t = [vad_id[k] for k in nna_idx]
        fa_delna = pd.DataFrame(fa_cmb_cc).dropna().values
        fa_total.append(fa_delna)

    return fa_total, nna_idx_t

In [ ]:
def fa_rtbk(start_t, end_t):
#未排序，上面是排序的
#输入样本期起始天和要预测的那天，即样本期末尾那天的后一天，输出一个list，list里有相应样本期长度对应个数的XY数组，及要预测那天的有效股票索引
    fa_total = []

    for t in range(start_t, end_t+1):
        fa_cmb = []
        vad_id = vad_index(t)
        for info in ml_names_list[end_t-1]:            
            fa_vec = globals()[info][vad_id, t]
            fa_vec_df = pd.DataFrame(fa_vec)
            fa_vec_arr = fa_vec_df.values
            nna_len = len(fa_vec_df.dropna())
            fa_vec_rank_arr = fa_vec_arr
            fa_cmb.append(fa_vec_rank_arr.T[0])          
        fa_cmb.append(stock_rt[vad_id, t+1])
        
        fa_cmb_cc = np.concatenate(fa_cmb).reshape((len(ml_names_list[end_t-1])+1), 
                                                   len(vad_id)).T
        if t == end_t:
            na_idx = np.unique(np.where(np.isnan(fa_cmb_cc))[0])
            nna_idx = sorted(list(set(np.arange(len(vad_id))).difference(set(na_idx))))
            nna_idx_t = [vad_id[k] for k in nna_idx]
        fa_delna = pd.DataFrame(fa_cmb_cc).dropna().values
        fa_total.append(fa_delna)

    return fa_total, nna_idx_t

In [ ]:
pd.DataFrame(factor_cat[60])

In [ ]:
def rfc(factor_sp, factor_test, n_est):# Random Tree Forest Model
    rf = RandomForestClassifier(n_estimators=n_est, 
                                random_state=1, 
                                n_jobs=4)
    rf.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = rf.predict_proba(factor_test[:, :-1])
    class_ord = rf.classes_
    fa_importance = rf.feature_importances_
    return y_test_pred, class_ord, fa_importance


def sp_testmat(factor_cat_p):
    factor_sp = np.concatenate(factor_cat_p[: -1])
    factor_cat_sp = factor_cat[-1]
    return factor_sp, factor_cat_sp

In [ ]:
def bys_b(factor_sp, factor_test, n_est):# Naive Bayes Model-Bernoulli
    by = BernoulliNB(alpha=1.0, binarize=0.0, 
                     class_prior=None, fit_prior=True)
    by.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = by.predict_proba(factor_test[:, :-1])
    class_ord = by.classes_
    return y_test_pred, class_ord


def bys_m(factor_sp, factor_test, n_est):# Naive Bayes Model-Polynomial
    by = MultinomialNB(alpha=1.0, class_prior=None, 
                       fit_prior=True)
    by.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = by.predict_proba(factor_test[:, :-1])
    class_ord = by.classes_
    return y_test_pred, class_ord


def bys_g(factor_sp, factor_test, n_est):# Naive Bayes Model-Gaussian
    by = GaussianNB(priors=None)
    by.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = by.predict_proba(factor_test[:, :-1])
    class_ord = by.classes_
    return y_test_pred, class_ord

In [ ]:
def lgbm(factor_sp, factor_test, n_est):#LightGBM model
    lgb =  LGBMClassifier(boosting_type='gbdt', objective= 'binary', 
                          n_jobs= 4, n_estimators=100, num_leaves= 31, 
                          max_depth= -1, learning_rate=0.1)
    lgb.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = lgb.predict_proba(factor_test[:, :-1])
    class_ord = lgb.classes_
    fa_importance = lgb.feature_importances_
    return y_test_pred, class_ord, fa_importance

In [ ]:
def l_rg(factor_sp, factor_test, n_est):# Logistic Model
    lr =  LogisticRegression(solver='saga', n_jobs= 4)
    lr.fit(factor_sp[:, :-1], factor_sp[:, -1])
    y_test_pred = lr.predict_proba(factor_test[:, :-1])
    class_ord = []
    return y_test_pred, class_ord

In [ ]:
def ratio1(factor_cat, r1):
    factor_cat_copy = copy.deepcopy(factor_cat)
    factor_cat_p = []
    for i in range(len(factor_cat_copy)):
        factor_cat_cp = factor_cat_copy[i]
        sort_idx = np.argsort(-factor_cat_cp[:, -1])
        top_idx = sort_idx[:int(r1 * len(sort_idx))]
        bottm_idx = sort_idx[-int(r1 * len(sort_idx)):]
        factor_cat_cp[top_idx, -1] = 0
        factor_cat_cp[bottm_idx, -1] = 1
        factor_cat_p.append(factor_cat_cp[list(top_idx)+list(bottm_idx), :])
    return factor_cat_p

In [ ]:
def ratio2(y_pred_list, r2):
# 构建股票策略选取的股票比例，y_pred_list即机器学习输出的预测结果，也就是概率矩阵
    y_pred_list_copy = copy.deepcopy(y_pred_list)
    mean_total = []
    corr_list = []
    top_idx_list = []
    bottm_idx_list = []
    for i in range(sld_len):
        prob_len = len(y_pred_list_copy[i][:, 0])
        factor_cat_sp2 = factor_test_list[i]
        prob_rt = np.concatenate([y_pred_list_copy[i][:,0], 
                                  factor_cat_sp2[:,-1]]).reshape(2, prob_len).T
        
        sort_idx = np.argsort(-prob_rt[:, 0])
        top_idx = sort_idx[:int(r2 * len(sort_idx))]
        bottm_idx = sort_idx[-int(r2 * len(sort_idx)):]
        top_idx_list.append(list(top_idx))
        bottm_idx_list.append(list(bottm_idx))

        top_mean = pd.DataFrame(prob_rt[top_idx, 1]).mean()
        bottm_mean = pd.DataFrame(prob_rt[bottm_idx, 1]).mean()
        mean_total.append(top_mean)
        mean_total.append(bottm_mean)
        
        concatvec = pd.DataFrame(prob_rt).dropna().values
        corr = spearmanr(concatvec[:,0],concatvec[:,1]).correlation
        corr_list.append(corr)
    
    mean_total_arr = np.array(mean_total).reshape(sld_len,2).T
    return mean_total_arr, corr_list, top_idx_list, bottm_idx_list

In [ ]:
def postion(nna_idx_list, top_idx_list, bottm_idx_list):
# 输出多空组合持仓数组及相应权重（等权的）
    posmat_list = []
    for i in range(sld_len):
        nna_idxp = nna_idx_list[i]
        top_idxp = top_idx_list[i]
        bottm_idxp = bottm_idx_list[i]
        top_len = len(top_idxp)
        bottm_len = len(bottm_idxp)
        posmat = np.zeros((len(stock_rt), 1))
        
        top_idx_org = [nna_idxp[k] for k in top_idxp]
        bottm_idx_org = [nna_idxp[k] for k in bottm_idxp]
        
        posmat[top_idx_org] = 1/top_len
        posmat[bottm_idx_org] = -1/bottm_len
        posmat_list.append(posmat)
    
    pos_arr = np.concatenate(posmat_list, axis=1)
#    pos_df = pd.DataFrame(posmat_arr, columns=[trd_dates[i][0] for i in range(end_t+1, end_t+1+sld_len)])
    return pos_arr

In [ ]:
def postion_buy(nna_idx_list, top_idx_list):
# 输出做多组合的持仓股票，标记为1，无权重
    posmat_list = []
    for i in range(sld_len):
        nna_idxp = nna_idx_list[i]
        top_idxp = top_idx_list[i]
#         bottm_idxp = bottm_idx_list[i]
        top_len = len(top_idxp)
#         bottm_len = len(bottm_idxp)
        posmat = np.zeros((len(stock_rt), 1))
        
        top_idx_org = [nna_idxp[k] for k in top_idxp]
#         bottm_idx_org = [nna_idxp[k] for k in bottm_idxp]
        
        posmat[top_idx_org] = 1
#         posmat[bottm_idx_org] = -1/bottm_len
        posmat_list.append(posmat)
    
    pos_arr = np.concatenate(posmat_list, axis=1)
#    pos_df = pd.DataFrame(posmat_arr, columns=[trd_dates[i][0] for i in range(end_t+1, end_t+1+sld_len)])
    return pos_arr

In [ ]:
fa_t = 240
fit_len = 40
sld_len = 2750-1-fit_len-fa_t#未取倒数第二天，即未预测最后一天收益
end_t = fa_t+fit_len-1# 60+fit_len-1
n_est = 30
r1 = 0.1


y_pred_list = []
class_ord_list = []
importance_list = []
nna_idx_list = []
factor_test_list = []

starttime = time.time()
for i in range(sld_len):
    endtime1 = time.time()
    sap_t1 = end_t-fit_len+1+i
    sap_t2 = end_t+1+i
    factor_cat, nna_idx, mlname = fa_rank_rtbk(sap_t1, sap_t2)
    nna_idx_list.append(nna_idx)
    
    factor_cat_p = ratio1(factor_cat, r1)
    factor_sp, factor_test = sp_testmat(factor_cat_p)
    factor_test_list.append(factor_test)
    y_test_pred, class_order, fa_importance = rfc(factor_sp, 
                                                  factor_test, n_est)

    y_pred_list.append(y_test_pred)
    importance_list.append(fa_importance)
    class_ord_list.append(class_order)
    endtime2 = time.time()
#    print('processed {} day, timelast: {} min'.format(i, ((endtime2-endtime1)/60)))
endtime3 = time.time()
print('timelast: {} min'.format((endtime3-starttime)/60))

In [ ]:
fa_t = 240
fit_len = 40
sld_len = 2750-1-fit_len-fa_t#未取倒数第二天，即未预测最后一天收益
end_t = fa_t+fit_len-1#  60+fit_len-1
n_est = 30
r1 = 0.15


y_pred_list = []
class_ord_list = []
importance_list = []
nna_idx_list = []
factor_test_list = []


starttime = time.time()
for i in range(sld_len):
    endtime1 = time.time()
    sap_t1 = end_t-fit_len+1+i
    sap_t2 = end_t+1+i
    factor_cat, nna_idx = fa_rank_rtbk(sap_t1, sap_t2)

    nna_idx_list.append(nna_idx)
    
    factor_cat_p = ratio1(factor_cat, r1)
    factor_sp, factor_test = sp_testmat(factor_cat_p)
    factor_test_list.append(factor_test)
    y_test_pred, class_order, fa_importance = lgbm(factor_sp, factor_test, n_est)

    y_pred_list.append(y_test_pred)
    importance_list.append(fa_importance)
    class_ord_list.append(class_order)
    endtime2 = time.time()
#    print('processed {} day, timelast: {} min'.format(i, ((endtime2-endtime1)/60)))
endtime3 = time.time()
print('timelast: {} min'.format((endtime3-starttime)/60))

In [ ]:
r2 = 0.01
mean_total_arr, corr_list, top_idx_list, bottm_idx_list = ratio2(y_pred_list, r2)

pos_arr = postion(nna_idx_list, top_idx_list, bottm_idx_list)
pos_df = pd.DataFrame(pos_arr, columns=[trd_dates[i][0] for i in range(end_t+1, end_t+1+sld_len)])
pos_df

In [ ]:
pos_df.to_csv('pos_df.csv')

In [ ]:
mean_cumsum = pd.DataFrame(mean_total_arr).T.cumsum().T

line = Line('cum_rtr')
arr = mean_cumsum.values
attr = [trd_dates[i] for i in range(end_t+1, end_t+sld_len+1)]
line.add('buy1set', attr, slide20(list(arr[0])), mark_line=['max'])
line.add('buy1sell1set', attr,  slide20(list(arr[0]-arr[1])), mark_line=['max'])
line.add('buy10set', attr, slide20(list(arr[1])), mark_line=['min'])
line

In [ ]:
corr_ml = corr_list
attr = [trd_dates[i] for i in range(end_t+1, end_t+sld_len+1)]
line = Line('IC')
line.add('ic', attr, slide20(corr_ml),mark_line=['average', 'max', 'min'])
line

In [ ]:
# 全时期选用的相应因子及其权重，并画出热力图
# select factors and corresponding weighht based on the whole time period and draw heatmap
factor_weight = pd.DataFrame(np.zeros((len(ml_names_all), stock_rt.shape[1])), 
                             index=ml_names_all)
for i in tnrange(sld_len):
    sap_t2 = end_t+1+i
    for j, info in enumerate(ml_names_list[sap_t2-1]):
        factor_weight.loc[info, sap_t2-1] = importance_list[i][j] / importance_list[i].sum()
        

ml_union_list = []
for i in tnrange(sld_len):
    starttime = time.time()
    sap_t2 = end_t+1+i
    ml_union_list.append(ml_names_list[sap_t2-1])
ml_union = sorted(list(np.unique(np.concatenate(ml_union_list))))

In [ ]:
factor_weight_s = 1000*factor_weight.loc[ml_union, end_t: end_t+sld_len-1]

x_axis = [trd_dates[end_t+i] for i in range(sld_len)]
y_axis = ml_union
data = [[i, j, factor_weight_s.iloc[j, i]] 
        for i in range(sld_len) 
        for j in range(factor_weight_s.shape[0])]
heatmap = HeatMap(width=1000, height=1600)
heatmap.add('factor_weight_heatmap', x_axis, y_axis, 
            data, is_visualmap=True,
            visual_text_color='#000', visual_orient='horizontal',
            calendar_cell_size=[0.5,0.5],
            visual_range_color=['#FFFFFF', '#FFE4B5','#FFA500'])
#            is_datazoom_show=True, datazoom_orient='vertical')
heatmap

In [ ]:
# 做多策略中行业股票数量的河流图
# stream graph to illustrate different industries of the stocks selcted in long strategy
posmat_buy = postion_buy(nna_idx_list, top_idx_list)
gfn1_code_s = pd.DataFrame(gfn1_code_index[:, end_t+1: end_t+1+sld_len])
gfn1_pos_buy = gfn1_code_s.values * posmat_buy
gfn1_code_num = np.zeros((11, gfn1_pos_buy.shape[1]))
gfn1_sum = gfn1_code_s.sum()
for i in range(gfn1_pos_buy.shape[1]):
    for j in range(11):
        gfn1_code_num[j, i] = len(np.where(gfn1_pos_buy[:,i] == j+1)[0]) / gfn1_sum[i]
        
gfn1_name = ['金融','房地产','周期行业','信息技术','可选消费','必选消费','医药生物','交通运输','公用事业','材料','工业']
data = []
for i in range(gfn1_code_num.shape[1]):
    for j in range(gfn1_code_num.shape[0]):
        data_p = []
        date = trd_dates[end_t+1+i][0]
        data_p.append(str(date)[:4]+'/'+str(date)[4:6]+'/'+str(date)[6:8])
        data_p.append(gfn1_code_num[j, i])
        data_p.append(gfn1_name[j])
        data.append(data_p)
        
data_fill = [[] for i in range((end_t+1)*11)] + data + [[] for i in range(2*11)]
year_end = []
for i in range(2008,2020):
    trd_t = np.where((trd_dates <= i*10000))[0][-1]
    year_end.append(trd_t)
year_end_f = [-1/11] + year_end

tr = ThemeRiver(width=800)
tr.add([gfn1_name[i] for i in range(gfn1_code_num.shape[0])], data, is_label_show=True,is_datazoom_show=True, yaxis_margin =20,yaxis_name_gap=100)
tr

In [ ]:
for k in range(len(year_end)):
    globals()['tr'+str(k)] = ThemeRiver(str(0)+str(k+7))
    globals()['tr'+str(k)].add([gfn1_name[i] for i in range(gfn1_code_num.shape[0])], 
                           data_fill[int(year_end_f[k]*11+1):year_end_f[k+1]*11+1], 
                           is_label_show=True, is_datazoom_show=True, datazoom_range=[0,25])

page = Page()
for k in range(len(year_end)):#len(year_end)
    page.add(globals()['tr'+str(k)])
page

In [ ]:
grid = Grid()

attr = ['LGBM', 'RF', 'NB', 'LR']
v1 = [26.18, 22.84, 15.91, 20.33]
v3 = [10.64, 8.84, 9.05, 7.33]
v4 = [68, 106, 90, 100]

bar1 = Bar(width=1200, height=600)
bar1.add("多空组合收益", attr, v1, yaxis_name="cum_rt")# long+short return
bar2 = Bar()
bar2.add("做多收益", attr, v3)# long return
bar3 = Bar()
bar3.add("时间", attr, v4, yaxis_name="time/min")



overlap = Overlap()
overlap.add(bar1)
overlap.add(bar2, is_add_yaxis=True, yaxis_index=0)
overlap.add(bar3, is_add_yaxis=True, yaxis_index=1)

grid.add(overlap, grid_right="10%")
grid